# xgboost

给定数据集，$\mathcal{D}\equiv\{{(\bf x}_n, y_n)\}$ 其中 ($n=1\cdots N$, $x_n\in R^M$, $y_n\in R$) ，并给定可微分的 loss function，其表达式记为 $L(\{\hat{y}_n\})\equiv \sum\limits_n l(y_n, \hat{y}_n)$，例如：

> 二分类问题中，常取 
$$y_n\in\{0, 1\}\quad 
l(y_n, y) \equiv -\left[y_n\ln\left({1\over 1+e^{-y}}\right) + (1-y_n)\ln\left(1-{1\over 1+e^{-y}}\right)\right]
=y_n\ln(1+e^{-y}) + (1-y_n)\ln(1+e^{y})$$
拟合问题中，常取
$$y_n\in R\quad, l(y_n, y) \equiv (y_n-y)^2$$

我们记 $g(y_n,y)\equiv \partial_yl(y_n,y),\quad h(y_n, y)\equiv \partial_y^2l(y_n,y)$, 二者的函数形式可“显式”的表达，例如：

> 拟合问题中 loss function 取上面的方差公式时，$$ g(y_n,y)=2(y-y_n),\quad h(y_n,y) = 2 $$

令 $${\hat y}^{(0)}({\bf x}) = f^{(0)} = {\rm argmin}_y L(\{\hat{y}_n=y\})={\rm argmin}_y \sum_il(y_n, y),\quad
g_n^{(0)}\equiv g(y_n,\hat{y}^{(0)}),\quad h_n^{(0)}\equiv h(y_n,\hat{y}^{(0)})$$
（注意第0步的 预测值 ${\hat y}^{(0)} $ 是一个常数，与输入 ${\bf x}$ 无关 ）， 例如：

> 拟合问题中 loss function 取上面的方差公式时，$$f^{(0)} = {\sum\limits_{n=1}^N  y_n\over N}$$ 是平均数。

假设在第$i$步 ($i=1,2\cdots$) 有一棵定义在 ${\bf x} \in R^D$ 树，记为 $f^{(i)}({\bf x})$, 其每个叶子$j$的输出值记为 $\{\omega_j \in R\}$ (其中 $j=1,2,\cdots, T$; $T$ 是叶子节点的个数) 

数据集 $\mathcal{D}\equiv\{{(\bf x}_n, y_n)\}$ 通过树 $f^{(i)}({\bf x})$ 后，落在叶子$j$ 的样本集，记为 $I_j^{(i)}$,

基于上述记号，对 $f^{(i)}({\bf x})$ 的每个节点 $j$ 我们定义如下变量：
$$G^{(i)}_j\equiv \sum_{n\in I^{(i)}_j}g_n^{(i-1)},\quad 
H^{(i)}_j\equiv \sum_{n\in I^{(i)}_j}h_n^{(i-1)},\quad 
S^{(i)}_j\equiv {\left(G^{(i)}_j\right)^2\over H^{(i)}_j +\lambda} 
$$
其中 $\lambda$ 是一个选定的和 L2$(\omega)$正则化相关的超参数。

我们将按如下步骤，生成第$i$步的目标树 $f^{(i)}({\bf x})$的结构：

> 以二叉树为例:
>
>（1）从一颗只有一个节点的树出发，该节点的分值记为 $S$.
>
>（2）将该节点分裂为两个子节点，得到一个新树，该树左右叶子的分值分别记为$S_L, S_R$；按一定方案遍历所有分裂方式，选择$S_L + S_R$ 总分最高的那颗树作为候选者
>
> (3) 当候选者的 $(S_L + S_R) > (S + \gamma$) 时，分裂有效，我们保留这个候选者作为新树 （其中 $\gamma$ 是一个选定的正则化参数，表示多出1个叶子节点所对应的“惩罚”）
>
> (4) 重复上述步骤，不断分裂新树的叶子节点，直到任何分裂都不能再使分值增加，或者达到其它树生长的约束条件（这些约束条件和“正则化”一样是为了避免overfitting，例如树不能太深，$I_j^{(i)}$ 元素数不能太少等）, 最终这颗不再“生长”的树，就是我们最终要找的“目标树”结构。

得到 $\{I_j^{(i)}\}$ 后，每个叶子 $j$ 的输出值定义为：

$$\omega_j \equiv -{G^{(i)}_j\over H^{(i)}_j +\lambda}$$

树的分裂方式 和 输出值 $\{\omega_j\}$ 锁定了目标树 $f^{(i)}({\bf x})$ 的完整定义，至此我们求出了$f^{(i)}({\bf x})$。

得到 第i步的 “boost” $f^{(i)}({\bf x})$ 后，我们按如下方式，找到 第i步的预测表达式

$${\hat y}^{(i)}({\bf x})\equiv \hat{y}^{(i-1)}({\bf x}) + f^{(i)}({\bf x})$$

将样本点${\bf x}_n$ 的预测值 $\hat{y}^{(i)}({\bf x}_n)$ 简记为 $\hat{y}^{(i)}_n$, 则上述迭代算法中
$ g_n^{(i)}\equiv g(y_n,\hat{y}^{(i)}_n),\quad h_n^{(i)}\equiv h(y_n,\hat{y}^{(i)}_n)$